In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import import_ipynb #requires pip install
from joblib import dump, load
import random

Import new dataset

In [41]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
anime = pd.read_csv(r"D:\dataset\encoding\anime_frame.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
#Function to turn "genres" columns from string into list
def string_to_list(genres):
    return eval(genres)

users["fav_genres"] = users["fav_genres"].apply(string_to_list)
anime["genre"] = anime["genre"].apply(string_to_list)
ratings["genre"] = ratings["genre"].apply(string_to_list)

In [42]:
#Build genre list
def split(genres):
    return str(genres).split(",")

def build_genre_list(dataframe):
    #Updating the set to get the individual genre names in the set instead of the original list of genres
    genre_set = set()
    for i in dataframe: 
        #print(i)
        genre_set.update(i)

    result = list(genre_set)
    #print(genre_set)
    #Removing blank spaces
    final_result = []
    for i in result:
        if i!="nan":
            #new = i.replace(" ", "")
            new = i.strip()
            final_result.append(new)

    #Removing duplicates now that spaces have been removed
    genre_list = list(set(final_result))
    return genre_list

Vectorize users records

In [43]:


user_cols = users.loc[:, ["username", "experience","gender", "generation",  "fav_anime_period"]]

encoder = LabelEncoder()
#Function to create a separate feature for each column and vectorize
def genre_columns():

    user_genres = users.loc[:, ["username", "fav_genres"]]
    genre_list = build_genre_list(anime["genre"])
    #print(genre_list)
    for i in genre_list:
        user_cols[i] = 0
    #only using genres that are in users favourites rather than full list of anime genres
    for x, y in user_genres.iterrows():
        user_row = user_cols.loc[user_cols["username"] == y["username"]]
        #print(y)
        #encode each of the three favourite genres with value: 1
        user_row[y["fav_genres"][0]] = 1 #experiement with changing to 10
        user_row[y["fav_genres"][1]] = 1
        user_row[y["fav_genres"][2]] = 1

        user_cols.loc[user_cols["username"] == y["username"]] = user_row

    genre_cols = user_cols.drop(columns=["experience","gender", "generation","fav_anime_period"])
    return genre_cols

#Vectorise features of a single user
def vectorize_single(user):
    #Get row of one user
    user_row = user_cols.loc[user_cols["username"] == user]
    user_vector = user_row.copy()
    #print(user_row)
    #Fit the encoder with the data of original df: user_cols
    exp = ["Newbie","Regular","Active","Veteran"]
    gender = ["Male", "Female", "Non-Binary"]
    gen = ["Gen-Alpha","Gen-Z","Millenials","Gen-X"]
    fav_period = ["New-Gen","Classic"]
    cols = [exp, gender, gen, fav_period]
    names = ["experience","gender","generation","fav_anime_period"]
    #for col in user_cols.columns:
    #    enc = encoder.fit(user_cols[col])
        #print(list(enc.classes_))
    #    user_vector[col]=enc.transform(user_vector[col]) #encode each column of target user
    for i in range (0,4):
        enc = encoder.fit(cols[i])
        #print(cols[i])
        current_col = names[i]
        #save the enconder to file
        dump(enc, f"{names[i]}.pkl")
        #print(list(enc.classes_))
        user_vector[current_col]=enc.transform(user_vector[current_col]) #encode each column of target user

    return user_vector

#Vectorise features of all users in the dataframe
def vectorize_all():
    all_users = user_cols.copy()

    for user in user_cols["username"]:
        user_vector = vectorize_single(user)
        all_users.loc[all_users["username"] == user] = user_vector
    #Make sure username field is in string form for identifier    
    all_users["username"] = user_cols["username"]
    #K-Nearest Neighbours
    #nb = KNeighborsClassifier(n_neighbors=5)
    return all_users

#Construct final vector of encoded features
def construct_vectors():
    string_features = vectorize_all()
    genre_features = genre_columns()
    final_vector = pd.merge(string_features, genre_features, on = "username")
    return final_vector

#result = construct_vectors()
#result.to_csv(r"D:\dataset\encoding\user_vectors.csv", index=False)
#genre_columns()
#build_genre_list(anime["genre"])
#vectorize_all()
#user_cols


C:\Users\titot\AppData\Local\Temp\ipykernel_26028\2655015919.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_row[y["fav_genres"][0]] = 1 #experiement with changing to 10
C:\Users\titot\AppData\Local\Temp\ipykernel_26028\2655015919.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_row[y["fav_genres"][1]] = 1
C:\Users\titot\AppData\Local\Temp\ipykernel_26028\2655015919.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [44]:
pd.set_option('display.max_columns', 500)
#result


Vectorise new user data

In [51]:
#test = load("fav_anime_period.pkl").transform(["New-Gen"])[0]
#print(test)

def new_user():
    username = random.randint(0,1000000)
    experience = load("encoders/experience.pkl").transform(["Veteran"])[0]
    gender = load("encoders/gender.pkl").transform(["Male"])[0]
    generation = load("encoders/generation.pkl").transform(["Millenials"])[0]
    fav_anime_period = load("encoders/fav_anime_period.pkl").transform(["Classic"])[0]
    fav_genres = ["Shounen","Comedy","Romance"]

    #Vectorise genres
    
    descriptive_cols = pd.DataFrame({"username":[username], "experience":[experience], "gender":[gender],"generation":[generation], "fav_anime_period":[fav_anime_period]})
    columns = ['username','Shounen', 'Seinen', 'Mystery', 'Kids', 'Parody', 'Josei', 'Shoujo', 'Fantasy', 'Sports', 'School', 'Action', 'Drama', 'Supernatural', 'Ecchi', 'Martial Arts', 'Horror', 'Sci-Fi', 'Adventure', 'Game', 'Cars', 'Romance', 'Police', 'Super Power', 'Space', 'Music', 'Military', 'Historical', 'Harem', 'Psychological', 'Samurai', 'Magic', 'Thriller', 'Slice of Life', 'Mecha', 'Vampire', 'Shounen Ai', 'Comedy']
    genres = pd.DataFrame(columns=columns)
    #p = {genre: 0 for genre in columns}
    genres = pd.DataFrame(columns=columns)
    genres.loc["test"]=0
    genres["username"]=username
    row = [0]*len(columns)
    row.insert(0, username)

    for y in fav_genres:
        #encode each of the three favourite genres with value: 1
        genres[y] = 1

    v = pd.merge(descriptive_cols, genres)
    print(len(columns))
    return v

sample_data = []
new_user()


38


,username,experience,gender,generation,fav_anime_period,Shounen,Seinen,Mystery,Kids,Parody,Josei,Shoujo,Fantasy,Sports,School,Action,Drama,Supernatural,Ecchi,Martial Arts,Horror,Sci-Fi,Adventure,Game,Cars,Romance,Police,Super Power,Space,Music,Military,Historical,Harem,Psychological,Samurai,Magic,Thriller,Slice of Life,Mecha,Vampire,Shounen Ai,Comedy
0,259697,3,1,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [46]:
vectorize_single("badking95")
#user_cols

,username,experience,gender,generation,fav_anime_period,Sci-Fi,Demons,Supernatural,Magic,Romance,Slice of Life,Parody,Shoujo Ai,Shoujo,Ecchi,Martial Arts,Horror,Police,Thriller,Space,Josei,Adventure,Shounen Ai,Comedy,Music,Harem,Mecha,Shounen,Drama,Historical,Action,Seinen,Sports,Cars,Fantasy,Kids,Psychological,Super Power,Game,Samurai,School,Military,Mystery,Vampire,Dementia
9301,badking95,3,1,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:

anime_vector = anime.loc[:, ["airing","studio", "genre", "time_period", "fame"]]
anime_vector
#anime
#anime_genres = users.loc[:, "genre"]
#genre_list = build_genre_list(anime_genres)
#for i in genre_list:
            #user_cols[i] = 0

,airing,studio,genre,time_period,fame
0,False,David Production,"[Comedy, Supernatural, Romance, Shounen]",New-Gen,famous
1,False,Gonzo,"[Comedy, Parody, Romance, School, Shounen]",Classic,semi-famous
2,False,Satelight,"[Comedy, Magic, School, Shoujo]",Classic,well-known
3,False,Hal Film Maker,"[Comedy, Drama, Magic, Romance, Fantasy]",Classic,semi-famous
4,False,J.C.Staff,"[Comedy, Drama, Romance, Shounen]",New-Gen,semi-famous
...,...,...,...,...,...
3235,False,Millepensee,"[Music, Drama]",New-Gen,niche
3236,False,Production I.G,"[Comedy, Sports, School, Shounen]",New-Gen,famous
3237,False,"Satelight, Production Reed","[Action, Mecha, Romance, Super Power, Supe...",Classic,well-known
3238,False,Toei Animation,"[Adventure, Fantasy]",Classic,unknown


In [48]:
rf = pd.read_csv(r"D:\dataset\encoding\collab_frame.csv")

full_ratings = rf[rf["username"].isin(users["username"])] #ensure that only ratings from users that are in the users dataframe are included

print(len(full_ratings))

1626920


Build user-item matrix with ratings (collaborative filtering)

In [ ]:

#not using genres because this should focus on ratings 
anime_matrix = pd.DataFrame(columns = users["username"].values)
anime_matrix["anime_id"] = anime["anime_id"]
anime_matrix.set_index("anime_id")
col = anime_matrix.pop("anime_id")
anime_matrix.insert(0, col.name, col)
#anime_matrix.iloc[:]=0
rating_matrix = pd.DataFrame(0, columns=anime_matrix.columns, index=anime_matrix.index) #set all values to 0 by default
rating_matrix["anime_id"] = anime["anime_id"]

#Filter rows in rating_matrix to only include anime that are also in the full_ratings table
rating_matrix = rating_matrix[rating_matrix["anime_id"].isin(full_ratings["anime_id"])]

for show in rating_matrix.iterrows():
    show_id = int(show[1]["anime_id"])
    #show_id = int(full_anime.loc[full_anime["title"]==show[0]]["anime_id"])
    anime_ratings = full_ratings.groupby("anime_id").get_group(show_id)
    usernames = list(anime_ratings["username"])
    scores = list(anime_ratings["my_score"]) #normalise ratings
    #print(anime_ratings)
    mean = list(anime_ratings["mean_score"])
    #mean = float(users.loc[users["username"]==users["stats_mean_score"]])
    norm = np.subtract(scores, mean)
    #print(norm)
    #print(usernames)
    #print(scores)      

    rating_matrix.loc[rating_matrix["anime_id"]==show_id,usernames] = scores #test with raw scores and normalised scores
#adding extra features to be used in combination with ratings for each show
extra_features = anime.loc[:, ["anime_id", "type", "studio", "time_period", "fame"]]
rating_matrix = rating_matrix.merge(extra_features, on="anime_id")

#Label encode the extra features
collab_encoder = LabelEncoder()
for col in extra_features.columns[1:]:
    col_fit = collab_encoder.fit(rating_matrix[col])
    rating_matrix[col] = col_fit.transform(rating_matrix[col])

rating_matrix.to_csv(r"D:\dataset\encoding\collab_scores.csv", index = False)
rating_matrix
#extra_features
